## LIBRARIES

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os
import seaborn as sns
from time import time
from joblib import dump
import umap
from sklearn.neighbors import KNeighborsClassifier
from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.multiclass import OneVsOneClassifier
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier

## STEP 0

In [ ]:
PatientsFolder = os.path.join(current_dir, 'Folder1')

ExcelData=pd.read_excel('Patients.xlsx')

PatientsExcel=list(ExcelData['Patient'].values)

CommonPatients=list(ExcelData['Patient'][ExcelData.loc[ExcelData['Phenotype']=='COMMON'].index].values)

R_patients=list(ExcelData['Patient'][ExcelData.loc[ExcelData['Relapse']=='YES'].index].values)
R_patients=list(set(R_patients)&set(CommonPatients))
NR_patients=list(ExcelData['Patient'][ExcelData.loc[ExcelData['Relapse']=='NO'].index].values)
NR_patients=list(set(NR_patients)&set(CommonPatients))

RegeneratedBM=['HNJ_020_t7.fcs','HNJ_023_t4.fcs', 'HNJ_031_t3.fcs', 'HNJ_052_t3.fcs','HNJ_059_t3.fcs', 'HNJ_071_t3.fcs','HNJ_072_t4.fcs','HNJ_076_t3.fcs','HNJ_081_t3.fcs','HNJ_098_t3.fcs','HNJ_101_t3.fcs'] 

def lecturachange(datafile):
    tube=flow.Tube(file=datafile)
    namemetadata=flow.operations.import_op.autodetect_name_metadata(datafile)
    import_op = flow.ImportOp(tubes = [tube],name_metadata='$PnN')
    try: 
        exp=import_op.apply()
    except KeyError:
        import_op = flow.ImportOp(tubes = [tube],name_metadata='$PnS')
        exp=import_op.apply()
    dicti=exp.metadata['fcs_metadata'][datafile]
    exportation=[];
    npar=dicti['$PAR'];
    dicti['list']=[]
    for i in range(1,npar+1):
        if '$P'+str(i)+'S'in dicti:
            aux=dicti['$P'+str(i)+'S'];
            if aux.isdecimal() or (aux[0].isdigit() and aux.endswith(('A','B','C'))):
                dicti['list'].append(['CD'+aux,dicti['$P'+str(i)+'N']])
            else:
                dicti['list'].append([aux,dicti['$P'+str(i)+'N']])
        else:
            dicti['list'].append([dicti['$P'+str(i)+'N']])
    list=dicti['list']
    for j in range(0,len(list)):
        markers=list[j];
        if len(markers)>1 and markers[0] not in exp.channels:
            exp.metadata[markers[0]]=exp.metadata[markers[1]]
            exp.data=exp.data.rename(columns={markers[1]:markers[0]})        

            del exp.metadata[markers[1]]
    return exp

def load_data(folder):
    experiment=[]
    
    with os.scandir(folder) as files:
        files = [file.name for file in files if (file.is_file() and file.name.endswith('.fcs'))and file.name[:7] in PatientsExcel]
       
        files.sort()
    for i in range(0,len(files)):
        myDir1=PatientsFolder+files[i]
        myDir2=PatientsFolder+'FILES/'+files[i] 

        exp=lecturachange(myDir1)
        exp.data.to_csv(myDir2)
        experiment.append(pd.read_csv(myDir2))
    
    print(str(len(files))+' files have been loaded. ')
    return [experiment,files]

start_time = time()
[CytoData,Files]=load_data(PatientsFolder)

elapsed_time = time() - start_time    
print('Time = '+str(elapsed_time))


for i in range(0,len(CytoData)):
    if len(CytoData[i])>=10000:
        CytoData[i] = CytoData[i].sample(n=10000,random_state=1)
    else: print('File '+Files[i]+' has less than 10000 cells')
    CytoData[i]=CytoData[i].sort_index(ignore_index=True).drop(['Unnamed: 0'],axis=1)    

aux=[]
for j in range(0,len(CytoData)):
    aux.append([Files[j],CytoData[j]])
experiment=aux

def patientsinfolder(exp):
    namespat=[]
    for i in range(len(exp)):
        a=exp[i][0][:7]
        if a not in namespat:
            namespat.append(a)
    return namespat

def patientsfiles(patient):
    aux=[]
    for i in Files:
        if i[:7]==patient:
            aux.append(i)
    return aux

def dataf(patient):
    df=[]
    for i in patientsfiles(patient):
        
        ColorNumber=float(Files.index(i))
        
        time=i[8:10]
               
        if i in RegeneratedBM:
            status='Regenerated'
        else:status='NonRegenerated'
        
       
        if patient in R_patients:
            RNR='R'
        elif patient in NR_patients:
            RNR='NR'
        else: RNR='Not common type'
        
        df.append(CytoData[Files.index(i)].assign(name=patient,time=time,ColorNumber=ColorNumber,RNR=RNR,Status=status))
        
    return df

def TotalDF(experiment):
    
    biomarkers=dataf(patientsinfolder(experiment)[0])[0].columns.values.tolist()
    for i in patientsinfolder(experiment):
        for j in range(len(dataf(i))):
            col=dataf(i)[j].columns.values.tolist()
            biomarkers=list(set(col).union(set(biomarkers)))
            
    TDF=pd.DataFrame(columns=biomarkers)

    for patient in patientsinfolder(experiment):
        TDF=TDF.append(dataf(patient)).fillna(np.nan).sort_index(ignore_index=True)
    
    return TDF

TDF=TotalDF(experiment)

TDF['name'][TDF['RNR']=='Not common type'].unique()

BasicBiomarkers=['CD10','CD20','CD45','CD19','CD34', 'SSC.A']
BasicColumns=BasicBiomarkers+['name','tiempo','ColorNumber','RNR']

todrop=list(TDF['name'][TDF['RNR']=='Not common type'].unique())

for patient in patientsinfolder(experiment):
    for t in range(len(patientsfiles(patient))):
        time=dataf(patient)[t]['time'][0]
        a=set(BasicBiomarkers)-set(dataf(patient)[t])
        
        if a!=set({}):
            todrop.append(patient)
            print('Patient '+ patient+' in '+ time + ' is missing column: '+ str(a))

patients0=set(patientsinfolder(experiment))-set(todrop)

print('\n Finally, '+str(len(patients0))+' patients are included in this work: \n \n',
      len(list(set(patients0)&set(R_patients))),' Relapse: \n',sorted(list(set(patients0)&set(R_patients))),'\n\n', 
      len(list(set(patients0)&set(NR_patients))),' Non Relapse: \n',sorted(list(set(patients0)&set(NR_patients))))

MyPatients=patients0
for i in TDF['name'].unique():
    if i not in MyPatients:
        print(i+' eliminated')
        TDF.drop(TDF[(TDF['name']==i)].index, inplace=True)
TDF=TDF.reset_index(drop=True)

sorted(TDF['name'][TDF['time']=='t0'].unique())==sorted(TDF['name'][TDF['time']=='t1'].unique())==sorted(TDF['name'][TDF['time']=='t2'].unique())

TDF.to_csv('cells_info.csv', index=False)  

## STEP 1

In [ ]:
current_dir = os.getcwd() 
PatientsFolder = os.path.join(current_dir, 'Folder1')
ImagesFolder = os.path.join(PatientsFolder, 'Images')

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"created folder: {folder_path}")
    else:
        print(f"The folder already exists: {folder_path}")

create_folder_if_not_exists(PatientsFolder)
create_folder_if_not_exists(ImagesFolder)

TDF = pd.read_csv('cells_info.csv')
TDF=TDF.assign(TDFindex=TDF.index.values)
TDF.set_index("TDFindex", inplace=True)

BasicBiomarkers=['CD10','CD45','CD20','CD19','CD34', 'SSC.A']
BasicBiomarkersReg=['CD10','CD20','CD45','CD19','CD34', 'SSC.A','CD58', 'CD66']
Pacient=['name','time','ColorNumber','RNR']

BasicColumns=BasicBiomarkers+Pacient
BasicColumnsReg=BasicBiomarkersReg+Pacient

dfRec=TDF[BasicColumns].loc[TDF['RNR']=='R']
dfRec=dfRec.assign(TDFindex=dfRec.index.values)
dfRec.set_index("TDFindex", inplace=True)

dfNoRec1=TDF[TDF['RNR']=='NR']
dfNoRec=dfNoRec1[BasicColumns].loc[dfNoRec1['Status']=='NonRegenerated'] 
dfNoRec=dfNoRec.assign(TDFindex=dfNoRec.index.values)
dfNoRec.set_index("TDFindex", inplace=True)

dfReg=TDF[BasicColumnsReg].loc[TDF['Status']=='Regenerated']
dfReg=dfReg.assign(TDFindex=dfReg.index.values)
dfReg.set_index("TDFindex", inplace=True)

dfReg=dfReg[dfReg['name']!='HNJ_020']
dfReg=dfReg[dfReg['name']!='HNJ_023']


xtrain = dfReg[BasicBiomarkers].values
ytrain = dfReg['name'].values
n = 50
d = 0.01

trans = umap.UMAP(n_neighbors=n, min_dist=d, n_jobs=-1).fit(xtrain)
dfembReg = pd.DataFrame(trans.embedding_, columns=['Umap 0', 'Umap 1'], index=dfReg.index)
dfembReg['name'] = ytrain 
plt.figure(figsize=(10, 7))
u, inv = np.unique(dfReg['name'].values, return_inverse=True)

miscatter = plt.scatter(dfembReg['Umap 0'], dfembReg['Umap 1'], s=0.2, 
                        c=dfReg['ColorNumber'] / 10, cmap='Spectral', label=dfReg['name'])
plt.legend(miscatter.legend_elements(size=1)[0], u)
plt.title(f'UMAP regenerated BM n={n} d={d}', fontsize=14)
plt.grid(False)

output_image_path = os.path.join(ImagesFolder, f'UMAP_Regenerated_BM_n{n}_d{d}.png')
plt.savefig(output_image_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved in: {output_image_path}")



if 'Umap 0' in TDF.columns and 'Umap 1' in TDF.columns:
    TDF = TDF.drop(['Umap 0', 'Umap 1'], axis=1)

TDF = TDF.join(dfembReg[['Umap 0', 'Umap 1']], how='left')
TDF.to_csv('cells_info.csv', index=True)
print("Updated 'cells_info.csv' file with UMAP 0 and UMAP 1 columns.")


columnsV = ['CD19','CD45','CD34','SSC.A', 'CD10','CD66']
a = len(columnsV)
b = int(np.sqrt(a))

fig, axs = plt.subplots(a//b + (a % b > 0), b, figsize=(30, 30))
colorS = 'viridis' 

i = 0
for marker in columnsV:
    ax = axs[i//b, i%b] 
    scatter = ax.scatter(dfembReg['Umap 0'], dfembReg['Umap 1'], s=0.2, 
                         c=dfReg[marker], cmap=sns.color_palette(colorS, as_cmap=True))
    ax.grid(False)
    ax.set_title(f'Intensity of {marker}', fontsize=40)
    fig.colorbar(scatter, ax=ax)
    
    i += 1       

fig.tight_layout()
output_path = os.path.join(ImagesFolder, 'CanvasAllColumns_Regeneration.png')
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved in: {output_path}")


xtestRec=dfRec[BasicBiomarkers].values
ytestRec=dfRec['name'].values

%time test_embeddingRec = trans.transform(xtestRec)

dfembRec=pd.DataFrame(test_embeddingRec)
dfembRec=dfembRec.assign(TDFindex=dfRec.index.values)
dfembRec.set_index("TDFindex", inplace=True)



xtestNoRec=dfNoRec[BasicBiomarkers].values
ytestNoRec=dfNoRec['name'].values

%time test_embeddingNoRec = trans.transform(xtestNoRec)

dfembNoRec=pd.DataFrame(test_embeddingNoRec)
dfembNoRec=dfembNoRec.assign(TDFindex=dfNoRec.index.values)
dfembNoRec.set_index("TDFindex", inplace=True)




dataframes = {'UMAP_regenerated': dfembReg,'UMAP_RecReg': dfembRec,'UMAP_NoRecReg': dfembNoRec,'df_regenerated': dfReg,'df_RecReg': dfRec,'df_NoRecReg': dfNoRec}

df_regenerated = dataframes['df_regenerated']
df_recreg = dataframes['df_RecReg']
df_norecreg = dataframes['df_NoRecReg']

df_UMAPregenerated = dataframes['UMAP_regenerated']
df_UMAPrecregenerated = dataframes['UMAP_RecReg']
df_UMAPnorecregenerated = dataframes['UMAP_NoRecReg']

dump(dataframes, 'dataframes.joblib')


def pedir_parametros():
    a = int(input("Enter the number of rows (a): "))
    b = int(input("Enter the number of columns (b): "))
    return a, b

def generar_umap_loop(dfReg, BasicBiomarkers, marker, colorS, ImagesFolder):
    a, b = pedir_parametros()
    
    xtrain = dfReg[BasicBiomarkers].values
    ytrain = dfReg['name'].values
    u, inv = np.unique(dfReg['name'].values, return_inverse=True)

    fig, axs = plt.subplots(a, b, figsize=(30, 30))
    fig.text(0.04, 0.5, 'Regenerated UMAP Parameters Loop', va='center', ha='center', rotation='vertical', fontsize=30)

    i = 0
    for ni in [5, 10, 25, 50, 100]:
        for di in [0.01, 0.05, 0.09, 0.5, 0.9]:
            trans = umap.UMAP(n_neighbors=ni, min_dist=di, n_jobs=-1).fit(xtrain)
            dfembReg = pd.DataFrame(trans.embedding_, index=dfReg.index)

            sc = axs[i // b, i % b].scatter(
                dfembReg[0], dfembReg[1], s=0.2, c=dfReg[marker], cmap=sns.color_palette(colorS, as_cmap=True)
            )
            axs[i // b, i % b].grid(False)
            axs[i // b, i % b].set_title(f'n={ni} y d={di}', fontsize=15)
            i += 1

    plt.subplots_adjust(wspace=0, hspace=0)
    output_path = os.path.join(ImagesFolder, 'UMAP_regenerated_n_and_d_loop.png')
    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.close()

    print(f"Image saved at: {output_path}")
    

## STEP 2

In [ ]:
current_dir = os.getcwd()
PatientsFolder = os.path.join(current_dir, 'Folder2')
ImagesFolder = os.path.join(PatientsFolder, 'Images')

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"created folder: {folder_path}")
    else:
        print(f"The folder already exists: {folder_path}")

create_folder_if_not_exists(PatientsFolder)
create_folder_if_not_exists(ImagesFolder)

TDF = pd.read_csv('cells_info.csv')
TDF=TDF.assign(TDFindex=TDF.index.values)
TDF.set_index("TDFindex", inplace=True)

dataframes = load('dataframes.joblib')

df_regenerated = dataframes['df_regenerated']
df_recreg = dataframes['df_RecReg']
df_norecreg = dataframes['df_NoRecReg']

df_UMAPregenerated = dataframes['UMAP_regenerated']
df_UMAPrecregenerated = dataframes['UMAP_RecReg']
df_UMAPnorecregenerated = dataframes['UMAP_NoRecReg']


dfembReg = df_UMAPregenerated
dfembReg_numeric = dfembReg.select_dtypes(include=[np.number])
X = dfembReg_numeric.values

####--------------------------------------######
BasicBiomarkers=['CD10','CD45','CD20','CD19','CD34', 'SSC.A']
BasicBiomarkersReg=['CD10','CD20','CD45','CD19','CD34', 'SSC.A','CD58', 'CD66']
OtherBiomarkers=['CD10', 'CD19', 'CD20', 'CD22', 'CD34', 'CD38', 'CD45',
              'CD58', 'CD66', 'FSC.A', 'FSC.H', 'HLADR', 'SSC.A']
Pacient=['name','time','ColorNumber','RNR']

BasicColumns=BasicBiomarkers+Pacient
BasicColumnsReg=BasicBiomarkersReg+Pacient
OtherColumns=OtherBiomarkers+Pacient

colors = {'BLymphocyte': '#CF4F75', 'Myeloid': '#FCAA67', 'Erythroblast': '#7D387D', 'MonocyteT': '#6bbfc7', 'Noise':'grey'}



def dbscan_cluster_plot():
    eps_values = input("Enter the eps values ​​(separated by commas): ").split(',')
    min_samples_values = input("Enter the values ​​of min_samples (separated by comma):").split(',')

    eps_values = [float(eps) for eps in eps_values]
    min_samples_values = [int(min_samples) for min_samples in min_samples_values]

    X = dfembReg.select_dtypes(include=['number']).values

    fig_width = 5 * len(min_samples_values)
    fig_height = 5 * len(eps_values)
    plt.figure(figsize=(fig_width, fig_height))
    
    i = 1

    for eps in eps_values:
        for min_samples in min_samples_values:
            
            db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
            labels = db.labels_

            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            n_noise = list(labels).count(-1)

            X_scaled = StandardScaler().fit_transform(X)

            unique_labels = set(labels)

            plt.subplot(len(eps_values), len(min_samples_values), i)
            plt.title(f"eps: {eps}, min_samples: {min_samples}\nClusters: {n_clusters}, Noise: {n_noise}")

            for label in unique_labels:
                if label != -1:
                    plt.plot(X_scaled[labels == label, 0], X_scaled[labels == label, 1], '.', label=f'Cluster {label}', markersize=0.5)
                else:
                    plt.plot(X_scaled[labels == label, 0], X_scaled[labels == label, 1], '.', color='gray', label='Noise', markersize=0.5)

            plt.gca().set_xticks([]) 
            plt.gca().set_yticks([]) 

            i += 1

    plt.tight_layout()

    now = datetime.datetime.now()
    output_filename = f'Graph_eps_min_samples_{now.month}-{now.day}_at_{now.hour}.png'
    output_path = os.path.join(ImagesFolder, output_filename)

    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.close()
    print(f"Image saved at: {output_path}")
    
    
dbscan_cluster_plot()


X = dfembReg.select_dtypes(include=['number']).values

db = DBSCAN(eps=0.5, min_samples=25).fit(X)
labels = db.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

X_scaled = StandardScaler().fit_transform(X)

unique_labels = set(labels)
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

centroids = []
for label in unique_labels:
    if label != -1:
        cluster_points = X_scaled[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        centroids.append(centroid)

centroids = np.array(centroids)

dfembReg['label'] = labels
plt.figure(figsize=(10, 6))

plt.plot(X_scaled[labels == 0, 0], X_scaled[labels == 0, 1], 'r.', label='cluster 0')
plt.plot(X_scaled[labels == 1, 0], X_scaled[labels == 1, 1], 'b.', label='cluster 1')
plt.plot(X_scaled[labels == 2, 0], X_scaled[labels == 2, 1], 'g.', label='cluster 2')
plt.plot(X_scaled[labels == 3, 0], X_scaled[labels == 3, 1], 'y.', label='cluster 3')
plt.plot(centroids[:, 0], centroids[:, 1], '^', markersize=8, c='purple', label='centroids')
plt.legend(loc='best')
plt.grid(False)
plt.title(f"Estimated number of clusters: {n_clusters_}", fontsize=15)

now = datetime.datetime.now()
output_path = os.path.join(ImagesFolder, f'RegeneratedBM{now.month}-{now.day}_at_{now.hour}.png')
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"Image saved at: {output_path}")

markers = {'CD19': max, 'CD66': max, 'CD45': min}

clusters = {}

for marker, func in markers.items():
    intensities = [np.mean(df_regenerated[marker][dfembReg['label'] == clust]) for clust in range(4)]
    cluster = intensities.index(func(intensities))
    clusters[marker] = cluster

BLymphocyte_cluster = clusters['CD19']
Myeloid_cluster = clusters['CD66']
Erythroblast_cluster = clusters['CD45']
MonocyteT_cluster = list(set(range(4)) - {BLymphocyte_cluster, Myeloid_cluster, Erythroblast_cluster})[0]

C_LB = centroids[BLymphocyte_cluster]
C_Mi = centroids[Myeloid_cluster]
C_Er = centroids[Erythroblast_cluster]
C_MT = centroids[MonocyteT_cluster]

cluster_labels = {BLymphocyte_cluster: 'BLymphocyte', Myeloid_cluster: 'Myeloid', Erythroblast_cluster: 'Erythroblast', MonocyteT_cluster: 'MonocyteT'}

for cluster, label in cluster_labels.items():
    dfembReg.loc[dfembReg['label'] == cluster, 'label'] = label

dfembReg.loc[dfembReg['label'] == -1, 'label'] = 'Noise'
output_path_df = os.path.join(ImagesFolder, f'ClusteredData_{now.month}-{now.day}_at_{now.hour}.csv')
dfembReg.to_csv(output_path_df, index=False)

print(f"Data saved at: {output_path_df}")

color_values = dfembReg['label'].map(colors)

plt.figure(figsize=(10, 10))
plt.scatter(dfembReg['Umap 0'], dfembReg['Umap 1'], c=color_values, label=dfembReg['label'], s=3)
plt.grid(False)
plt.title('DBSCAN clustering', fontsize=15)

handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=10, label=label)
    for label, col in colors.items()
]
plt.legend(handles=handles, loc='upper right', fontsize=12)

plt.xticks([])  
plt.yticks([]) 
plt.xlabel('UMAP_0', fontsize=15)
plt.ylabel('UMAP_1', fontsize=15)

now = datetime.datetime.now()
output_filename = f'UMAP_and_DBSCAN_Regenerated_BM_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)

plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()
print(f"Image saved at: {output_path}")



markersReg = ['CD19', 'CD45', 'CD66']
a = len(markersReg)

fig, axs = plt.subplots(1, a, figsize=(33, 10))
colorS = 'winter' 

i = 0
for marker in markersReg:
    ax = axs[i]
    im = ax.scatter(dfembReg['Umap 0'], dfembReg['Umap 1'], s=4, c=df_regenerated[marker], cmap=sns.color_palette(colorS, as_cmap=True), vmin=0, vmax=1)
    ax.grid(False)
    ax.set_xticks([])  
    ax.set_yticks([])  
    ax.set_title('Intensity of ' + marker, fontsize=30)
    
    i += 1

fig.tight_layout()

fig.subplots_adjust(right=0.85)
cbar_ax = fig.add_axes([0.87, 0.1, 0.015, 0.8])  
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Intensity', fontsize=40)
cbar.set_ticks([0, 1]) 
cbar.ax.tick_params(labelsize=30) 

now = datetime.datetime.now()
output_filename = f'RegeneratedBM_Horizontal_intensities_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)

plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"Imagen guardada en: {output_path}")


df_UMAPrecregenerated.rename(columns={0: 'Umap 0', 1: 'Umap 1'}, inplace=True)
df_UMAPnorecregenerated.rename(columns={0: 'Umap 0', 1: 'Umap 1'}, inplace=True)

X_train = dfembReg[['Umap 0', 'Umap 1']]
y_train = dfembReg['label']
X_test_Rec = df_UMAPrecregenerated[['Umap 0', 'Umap 1']]
X_test_NoRec = df_UMAPnorecregenerated[['Umap 0', 'Umap 1']]
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_Rec_scaled = scaler.transform(X_test_Rec)
X_test_NoRec_scaled = scaler.transform(X_test_NoRec)

clf = OneVsOneClassifier(SVC(kernel='rbf', max_iter=20000, tol=1e-4))
clf.fit(X_train_scaled, y_train)

Y_result_Rec = clf.predict(X_test_Rec_scaled)
Y_result_NoRec = clf.predict(X_test_NoRec_scaled)

if len(Y_result_Rec) == len(df_UMAPrecregenerated):
    df_UMAPrecregenerated['label'] = Y_result_Rec

if len(Y_result_NoRec) == len(df_UMAPnorecregenerated):
    df_UMAPnorecregenerated['label'] = Y_result_NoRec

EMB_label = dfembReg.combine_first(df_UMAPrecregenerated).combine_first(df_UMAPnorecregenerated)
TDF_EMB = TDF.join(EMB_label, how='left', rsuffix='_emb')

train_colors = [colors.get(label, 'grey') for label in y_train]
rec_colors = [colors.get(label, 'grey') for label in Y_result_Rec]
norec_colors = [colors.get(label, 'grey') for label in Y_result_NoRec]

fig, ax = plt.subplots(figsize=(20, 20))
ax.scatter(X_train['Umap 0'], X_train['Umap 1'], c=train_colors, s=5, alpha=0.05, label='Train Data')
ax.scatter(X_test_Rec['Umap 0'], X_test_Rec['Umap 1'], c=rec_colors, s=5, label='Regenerated')
ax.scatter(X_test_NoRec['Umap 0'], X_test_NoRec['Umap 1'], c=norec_colors, s=5, label='Not Regenerated')

ax.grid(False)
ax.legend(loc='upper right', fontsize=12)

now = datetime.datetime.now()
output_filename = f'Vector_support_machine_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)

plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved in: {output_path}")


TDF_EMB_sin_nan = TDF_EMB.dropna(subset=['label', 'Umap 0','Umap 1'])
TDF_EMB= TDF_EMB[~((TDF_EMB['RNR'] == 'Sana'))]
dataframes['TDF_EMB_sin_nan'] = TDF_EMB_sin_nan
dataframes['TDF_EMB'] = TDF_EMB
dump(dataframes, 'dataframes.joblib')

def PairwiseMarkers(m1, m2):
    color_values = TDF_EMB['label'][TDF_EMB['Status'] == 'Regenerated'].map(colors)
        
    plt.figure(figsize=(5, 5))
    plt.scatter(
        TDF_EMB[TDF_EMB['Status'] == 'Regenerated'][m1], 
        TDF_EMB[TDF_EMB['Status'] == 'Regenerated'][m2], 
        c=color_values, 
        label=dfembReg['label'], 
        s=1.5
    )
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.title('Regenerated Bone Marrow', fontsize=15)
    handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=2, label=f'{label}') 
        for label, col in colors.items()
    ]
    plt.xlabel(m1, fontsize=15)
    plt.ylabel(m2, fontsize=15)

    now = datetime.datetime.now()
    output_filename = f'PairwiseMarkers{m1}_{m2}_{now.month}-{now.day}_at_{now.hour}.png'
    output_path = os.path.join(ImagesFolder, output_filename)

    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.close()

    print(f"Image saved at: {output_path}")
    
    
    
colorscmaps = {label: sns.light_palette(color, as_cmap=True) for label, color in colors.items()}   

def FollowUp(name, marker):
    j = 0
    times = TDF_EMB['time'][TDF_EMB['name'] == name].unique()
    a = len(times)

    subplot_x = 10
    subplot_y = 6
    total_width = subplot_x * a
    total_height = subplot_y

    fig, axs = plt.subplots(1, a, figsize=(total_width, total_height))
    fig.subplots_adjust(top=0.8, wspace=0.3)
    fig.suptitle(f'Patient {name} \n {marker} intensity\n', fontsize=30)

    for t in sorted(times):
        aux = TDF_EMB[(TDF_EMB['name'] == name) & (TDF_EMB['time'] == t)]
        
        for label in aux['label'].unique():
            sub_aux = aux[aux['label'] == label]

            if sub_aux[marker].max() - sub_aux[marker].min() == 0:
                norm_intensity = sub_aux[marker] * 0
            else:
                norm_intensity = (sub_aux[marker] - sub_aux[marker].min()) / (sub_aux[marker].max() - sub_aux[marker].min())
            
            cmap = colorscmaps.get(label, 'viridis') 
            axs[j].scatter(sub_aux['Umap 0_emb'], sub_aux['Umap 1_emb'], c=norm_intensity, cmap=cmap, label=label, s=3)  
        
        axs[j].grid(False)
        axs[j].set_title(str(t), fontsize=25)
        
        handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=10, label=label) for label, col in colors.items()]
        axs[j].legend(handles=handles, loc='upper left', fontsize=15)
        
        j += 1
        
    now = datetime.datetime.now()
    output_filename = f'FollowUp_{name}_BM_{marker}_{now.month}-{now.day}_at_{now.hour}-{now.minute}.png'
    output_path = os.path.join(ImagesFolder, output_filename)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    fig.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.close(fig) 
    
    print(f"Image saved at: {output_path}")
    
    
    
def RegeneratedBM(marker):
    subplot_x = 10
    subplot_y = 10
    total_width = subplot_x * 4
    total_height = subplot_x

    fig, axs = plt.subplots(1, 4, figsize=(total_width, total_height))
    fig.subplots_adjust(top=0.5)
    fig.suptitle('All Regenerated BM ' + marker + '\n', fontsize=30)    
    fig.tight_layout()

    if not isinstance(axs, np.ndarray):
        axs = np.array([axs])

    timepoints = ['t0', 't1', 't2', 't3']

    for j, t in enumerate(timepoints):  
        valid_names = list(set(TDF_EMB['name'][TDF_EMB['time'] == t].unique()) & set(df_regenerated.name.unique()))
        for name in sorted(valid_names):
            aux = TDF_EMB[(TDF_EMB['name'] == name) & (TDF_EMB['time'] == t)]
            
            for label in aux['label'].unique():
                sub_aux = aux[aux['label'] == label]
                
                norm_intensity = (sub_aux[marker] - sub_aux[marker].min()) / (sub_aux[marker].max() - sub_aux[marker].min())
                cmap = colorscmaps[label]
                
                axs[j].scatter(sub_aux['Umap 0_emb'], sub_aux['Umap 1_emb'], c=norm_intensity, cmap=cmap, label=label, s=1.5)
        
        axs[j].grid(False)
        axs[j].set_title(str(t), fontsize=25)
        handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=3, label=label) for label, col in colors.items()]
        axs[j].legend(handles=handles, loc='upper left', fontsize=15)

    now = datetime.datetime.now()
    output_filename = f'FollowUp_All_BM_{marker}_{now.month}-{now.day}_at_{now.hour}-{now.minute}.png'
    output_path = os.path.join(ImagesFolder, 'Regeneration', output_filename)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    fig.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.close(fig)

    print(f"Image saved at: {output_path}")

## STEP 3

In [ ]:
current_dir = os.getcwd()  
PatientsFolder = os.path.join(current_dir, 'Folder3')
ImagesFolder = os.path.join(PatientsFolder, 'Images')

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"created folder: {folder_path}")
    else:
        print(f"The folder already exists: {folder_path}")

create_folder_if_not_exists(PatientsFolder)
create_folder_if_not_exists(ImagesFolder)

TDF = pd.read_csv('cells_info.csv')
TDF=TDF.assign(TDFindex=TDF.index.values)
TDF.set_index("TDFindex", inplace=True)

dataframes = load('dataframes.joblib')
TDF_EMB = dataframes['TDF_EMB']
TDF_EMB_sin_nan = dataframes['TDF_EMB_sin_nan']

TDF_EMB = TDF_EMB.drop(columns=["Umap 0", "Umap 1"])
df_BLympho=TDF_EMB[TDF_EMB['label']=='BLymphocyte']
df_BLympho=df_BLympho.assign(TDFindex=df_BLympho.index.values)
df_BLympho.set_index("TDFindex", inplace=True)

df_Regenerated_BLympho=df_BLympho[df_BLympho['Status']=='Regenerated']

BasicBiomarkers=['CD10','CD45','CD20','CD19','CD34', 'SSC.A']
BasicColumns=BasicBiomarkers+['name','time','ColorNumber','RNR']

a = 6
b = 6

fig, axs = plt.subplots(a, b, figsize=(30, 30))

colorS = 'viridis'

i = 0
marker = 'CD10'

for ni in [10, 25, 50, 100, 150, 200]:
    for di in [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]:
        n = ni
        d = di
        
        xtrainBLympho = df_Regenerated_BLympho[BasicBiomarkers].values 
        ytrainBLympho = df_Regenerated_BLympho['name'].values
        
        trans = umap.UMAP(n_neighbors=n, min_dist=d, random_state=1, n_jobs=1).fit(xtrainBLympho)
        
        dfembReg_BLympho = pd.DataFrame(trans.embedding_)

        dfembReg_BLympho = dfembReg_BLympho.assign(TDFindex=df_Regenerated_BLympho.index.values)
        dfembReg_BLympho.set_index("TDFindex", inplace=True)

        u, inv = np.unique(df_Regenerated_BLympho.name.values, return_inverse=True)

        axs[i // b, i % b].scatter(dfembReg_BLympho[0], dfembReg_BLympho[1], 
                                   s=0.3,  
                                   c=df_Regenerated_BLympho[marker],
                                   cmap=sns.color_palette(colorS, as_cmap=True), 
                                   label=df_Regenerated_BLympho['name'])
        axs[i // b, i % b].grid(False)
        axs[i // b, i % b].set_title('n= ' + str(n) + '  d= ' + str(d), fontsize=15)
        axs[i // b, i % b].set_xticks([])
        axs[i // b, i % b].set_yticks([])
        i += 1

plt.subplots_adjust(wspace=0.05, hspace=0.05)

now = datetime.datetime.now()
output_filename = f'CD10_intensities_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved at: {output_path}")

xtrainBLympho = df_Regenerated_BLympho[BasicBiomarkers].values 
ytrainBLympho = df_Regenerated_BLympho['name'].values

n = 200
d = 0.01

trans = umap.UMAP(n_neighbors=n, min_dist=d, random_state=1).fit(xtrainBLympho)

dfembReg_BLympho = pd.DataFrame(trans.embedding_)
dfembReg_BLympho = dfembReg_BLympho.assign(TDFindex=df_Regenerated_BLympho.index.values)
dfembReg_BLympho.set_index("TDFindex", inplace=True)
dfembReg_BLympho = dfembReg_BLympho.rename(columns={0: 'UMAP_BLympho_0', 1: 'UMAP_BLympho_1'})

df_Regenerated_BLympho = df_Regenerated_BLympho.join(dfembReg_BLympho)

plt.figure(figsize=(10, 7))
u, inv = np.unique(df_Regenerated_BLympho.name.values, return_inverse=True)
plt.grid(False)
miscatter = plt.scatter(dfembReg_BLympho['UMAP_BLympho_0'], 
                        dfembReg_BLympho['UMAP_BLympho_1'], 
                        s=10,  
                        c=df_Regenerated_BLympho['ColorNumber'], 
                        cmap='Spectral', 
                        label=df_Regenerated_BLympho['name'])
plt.legend(miscatter.legend_elements(size=1)[0], u)
plt.title('Regenerated B Lymphocytes UMAP n=' + str(n) + ' y d= ' + str(d), fontsize=14)

now = datetime.datetime.now()
output_filename = f'RegeneratedBM_Horizontal_intensities_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved at: {output_path}")

BLympho_Biomarkers = ['CD19', 'CD45', 'CD10', 'CD34']
a = len(BLympho_Biomarkers)
b = int(np.sqrt(a))

fig, axs = plt.subplots((a + b - 1) // b, b, figsize=(20, 15))

i = 0
for marker in BLympho_Biomarkers:
    ax = axs[i // b, i % b]
    im = ax.scatter(df_Regenerated_BLympho['UMAP_BLympho_0'], 
                    df_Regenerated_BLympho['UMAP_BLympho_1'], 
                    s=10, 
                    c=df_Regenerated_BLympho[marker], 
                    cmap=sns.color_palette('plasma', as_cmap=True), 
                    vmin=-0.25, 
                    vmax=1.5)
    ax.grid(False)
    ax.set_xticks([])  
    ax.set_yticks([])
    ax.set_title('Intensity of ' + marker, fontsize=20)
    i += 1

fig.tight_layout()
fig.subplots_adjust(right=0.85)

cbar_ax = fig.add_axes([0.87, 0.15, 0.02, 0.7])  
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Intensity', fontsize=20)

now = datetime.datetime.now()
output_filename = f'Lymphocytes_UMAP_Markers_Intensity_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"Imagen guardada en: {output_path}")


markersReg = ['CD10', 'CD45', 'CD34']
a = len(markersReg)

fig, axs = plt.subplots(1, a, figsize=(33, 10))
colorS = 'winter'

i = 0
for marker in markersReg:
    ax = axs[i]
    im = ax.scatter(df_Regenerated_BLympho['UMAP_BLympho_0'], 
                    df_Regenerated_BLympho['UMAP_BLympho_1'], 
                    s=40, 
                    c=df_Regenerated_BLympho[marker], 
                    cmap=sns.color_palette(colorS, as_cmap=True), 
                    vmin=0, 
                    vmax=1)
    ax.grid(False)
    ax.set_xticks([])  
    ax.set_yticks([])
    ax.set_title('Intensity of ' + marker, fontsize=30)
    i += 1

fig.tight_layout()
fig.subplots_adjust(right=0.85)

cbar_ax = fig.add_axes([0.87, 0.1, 0.015, 0.8])  
cbar = fig.colorbar(im, cax=cbar_ax)
cbar.set_label('Intensity', fontsize=40)
cbar.set_ticks([0, 1]) 
cbar.ax.tick_params(labelsize=30) 

now = datetime.datetime.now()
output_filename = f'Regenerated_Lymphocytes_Horizontal_Markers_Intensity_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"imaged saved at: {output_path}")


df_BLympho_Rec=df_BLympho[df_BLympho['RNR']=='R']
df_BLympho_Rec=df_BLympho_Rec.assign(TDFindex=df_BLympho_Rec.index.values)
df_BLympho_Rec.set_index("TDFindex", inplace=True)

df_BLympho_NoRec1=df_BLympho[df_BLympho['RNR']=='NR']
df_BLympho_NoRec=df_BLympho_NoRec1.loc[df_BLympho_NoRec1['Status']=='NonRegenerated'] 
df_BLympho_NoRec=df_BLympho_NoRec.assign(TDFindex=df_BLympho_NoRec.index.values)
df_BLympho_NoRec.set_index("TDFindex", inplace=True)


xtestLymphoRec=df_BLympho_Rec[BasicBiomarkers].values
ytestLymphoRec=df_BLympho_Rec['name'].values 

%time test_embeddingLymphoRec = trans.transform(xtestLymphoRec)

dfembLymphoRec=pd.DataFrame(test_embeddingLymphoRec)
dfembLymphoRec= dfembLymphoRec.rename(columns={0: 'UMAP_BLympho_0', 1: 'UMAP_BLympho_1'})

dfembLymphoRec=dfembLymphoRec.assign(TDFindex=df_BLympho_Rec.index.values)
dfembLymphoRec.set_index("TDFindex", inplace=True)

df_BLympho_Rec=df_BLympho_Rec.join(dfembLymphoRec)

xtestLymphoNoRec=df_BLympho_NoRec[BasicBiomarkers].values
ytestLymphoNoRec=df_BLympho_NoRec['name'].values 

%time test_embeddingLymphoNoRec = trans.transform(xtestLymphoNoRec)

dfembLymphoNoRec=pd.DataFrame(test_embeddingLymphoNoRec)
dfembLymphoNoRec= dfembLymphoNoRec.rename(columns={0: 'UMAP_BLympho_0', 1: 'UMAP_BLympho_1'})

dfembLymphoNoRec=dfembLymphoNoRec.assign(TDFindex=df_BLympho_NoRec.index.values)
dfembLymphoNoRec.set_index("TDFindex", inplace=True)

df_BLympho_NoRec=df_BLympho_NoRec.join(dfembLymphoNoRec)

All_BLymphocytes=df_BLympho_NoRec.combine_first(df_BLympho_Rec)
All_BLymphocytes=All_BLymphocytes.combine_first(df_Regenerated_BLympho)
All_BLymphocytes=All_BLymphocytes.assign(TDFindex=All_BLymphocytes.index.values)
All_BLymphocytes.set_index('TDFindex',inplace=True)


dataframes = {}
dataframes['All_BLymphocytes'] = All_BLymphocytes
dump(dataframes, 'dataframes_BLymphocytes.joblib')





## STEP 4

In [ ]:
current_dir = os.getcwd()  
PatientsFolder = os.path.join(current_dir, 'Folder4')
ImagesFolder = os.path.join(PatientsFolder, 'Images')

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"created folder: {folder_path}")
    else:
        print(f"The folder already exists: {folder_path}")

create_folder_if_not_exists(PatientsFolder)
create_folder_if_not_exists(ImagesFolder)

TDF = pd.read_csv('cells_info.csv')
TDF=TDF.assign(TDFindex=TDF.index.values)
TDF.set_index("TDFindex", inplace=True)

dataframes1 = load('dataframes.joblib')
TDF_EMB = dataframes1['TDF_EMB']
dataframes = load('dataframes_BLymphocytes.joblib')
All_BLymphocytes = dataframes['All_BLymphocytes']

BasicBiomarkers=['CD10','CD45','CD20','CD19','CD34', 'SSC.A']
BasicColumns=BasicBiomarkers+['name','time','ColorNumber','RNR']

df_Regenerated_BLympho=All_BLymphocytes[All_BLymphocytes['Status']=='Regenerated']
df_Regenerated_BLympho=df_Regenerated_BLympho.assign(TDFindex=df_Regenerated_BLympho.index.values)
df_Regenerated_BLympho.set_index('TDFindex',inplace=True)


X = df_Regenerated_BLympho[['UMAP_BLympho_0', 'UMAP_BLympho_1']].values

db = DBSCAN(eps=0.6, min_samples=70).fit(X)
labels = db.labels_

n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

X = StandardScaler().fit_transform(X)

unique_labels = set(labels)
core_samples_mask = np.zeros_like(labels, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True

centroids = []
for label in unique_labels:
    if label != -1:  
        cluster_points = X[labels == label]
        centroid = np.mean(cluster_points, axis=0)
        centroids.append(centroid)

centroids = np.array(centroids)

df_Regenerated_BLympho = df_Regenerated_BLympho.copy()
df_Regenerated_BLympho.loc[:, 'Lympho_label'] = labels

plt.figure(figsize=(10, 6))
colors = ['r', 'b', 'g', 'y', 'c', 'm']  
for i, color in zip(unique_labels, colors):
    if i != -1:  
        plt.plot(X[labels == i, 0], X[labels == i, 1], '.', color=color, label=f'cluster {i}')
        
if len(centroids) > 0:
    plt.plot(centroids[:, 0], centroids[:, 1], '^', markersize=10, c='purple', label='centroids')

plt.grid(False)
plt.legend(loc='best')
plt.title(f"Estimated number of clusters: {len(centroids)}", fontsize=15)

now = datetime.datetime.now()
output_filename = f'RegeneratedBM_Horizontal_intensities_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"image saved at: {output_path}")

marker = 'CD45'
v = []
for clust in [0, 1, 2]:
    m = np.mean(df_Regenerated_BLympho[marker][df_Regenerated_BLympho['Lympho_label'] == clust])
    v.append(m)

v_array = np.array(v)
sorted_indices = np.argsort(v_array)[::-1]

names = ['Mature', 'Trans', 'ProBPreB']
clusternames = {sorted_indices[i]: names[i] for i in range(len(sorted_indices))}
df_Regenerated_BLympho['Lympho_label'] = df_Regenerated_BLympho['Lympho_label'].map(clusternames)


colors = {'Mature' : '#902747', 'Trans': '#E9AFC1', 'ProBPreB': '#A567CB',np.nan: 'grey'}
color_values = df_Regenerated_BLympho['Lympho_label'].map(colors)

plt.figure(figsize=(7, 7))
plt.scatter(df_Regenerated_BLympho['UMAP_BLympho_0'], df_Regenerated_BLympho['UMAP_BLympho_1'], c=color_values, label=df_Regenerated_BLympho['label'], s=10)
plt.grid(False)
plt.title('DBSCAN clustering', fontsize=15)

handles = [
    plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=6, label=f'{label}') 
    for label, col in colors.items()
]
plt.legend(handles=handles, loc='best', fontsize=12)

plt.xticks([])
plt.yticks([])
plt.xlabel('UMAP_BLympho_0', fontsize=15)
plt.ylabel('UMAP_BLympho_1', fontsize=15)

now = datetime.datetime.now()
output_filename = f'UMAP_and_DBSCAN_Regenerated_B_Lymphocytes_{now.month}-{now.day}_at_{now.hour}.png'
output_path = os.path.join(ImagesFolder, output_filename)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()

print(f"Image saved at: {output_path}")

def PairwiseLympho(m1, m2):
    colors = {'Mature': '#902747','Trans': '#E9AFC1','ProBPreB': '#A567CB',np.nan: 'grey'}

    color_values = df_Regenerated_BLympho['Lympho_label'].map(colors)
    plt.figure(figsize=(5, 5))

    plt.scatter(df_Regenerated_BLympho[m1], df_Regenerated_BLympho[m2], c=color_values, label=df_Regenerated_BLympho['Lympho_label'], s=2)
    plt.grid(False)
    plt.xticks([])  
    plt.yticks([])

    plt.title('Regenerated LymphoB', fontsize=15)
    handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=3, label=f'{label}') 
        for label, col in colors.items()
    ]
    plt.xlabel(m1, fontsize=15)
    plt.ylabel(m2, fontsize=15)

    now = datetime.datetime.now()
    output_filename = f'Regenerated_B_Lymphocytes_{m1}_vs_{m2}_{now.month}-{now.day}_at_{now.hour}_{now.minute}.png'
    output_path = os.path.join(ImagesFolder, output_filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    print(f"Image saved at: {output_path}")
    
df_Regenerated_BLympho['Lympho_label'].fillna('Unknown', inplace=True)
colors = {'Mature': '#902747', 'Trans': '#E9AFC1', 'ProBPreB': '#A567CB', np.nan: 'grey', 'Unknown': 'grey'}
color_values = df_Regenerated_BLympho['Lympho_label'].map(colors)

X_train = df_Regenerated_BLympho[['UMAP_BLympho_0', 'UMAP_BLympho_1']]
y_train = df_Regenerated_BLympho['Lympho_label']
X_test_Rec = All_BLymphocytes[All_BLymphocytes['RNR'] == 'R'][['UMAP_BLympho_0', 'UMAP_BLympho_1']]

aux = All_BLymphocytes[All_BLymphocytes['Status'] == 'NonRegenerated']
aux = aux.assign(TDFindex=aux.index.values)
aux.set_index('TDFindex', inplace=True)
X_test_NoRec = aux[aux['RNR'] == 'NR'][['UMAP_BLympho_0', 'UMAP_BLympho_1']]

clf = OneVsOneClassifier(SVC(kernel='rbf', max_iter=10000)).fit(X_train, y_train)
Y_result_Rec = clf.predict(X_test_Rec)
Y_result_NoRec = clf.predict(X_test_NoRec)

fig, ax = plt.subplots(figsize=(20, 20))
ax.scatter(X_train['UMAP_BLympho_0'], X_train['UMAP_BLympho_1'], c=color_values, label=df_Regenerated_BLympho['Lympho_label'], s=2)

ax.scatter(
    X_test_Rec['UMAP_BLympho_0'], 
    X_test_Rec['UMAP_BLympho_1'], 
    c=[colors[v] for v in Y_result_Rec], 
    s=5
)

ax.scatter(
    X_test_NoRec['UMAP_BLympho_0'], 
    X_test_NoRec['UMAP_BLympho_1'], 
    c=[colors[v] for v in Y_result_NoRec], 
    s=5
)

ax.grid(False)

now = datetime.datetime.now()
output_filename = f'Lymphocyte_Classification_{now.month}-{now.day}_at_{now.hour}_{now.minute}.png'
output_path = os.path.join(ImagesFolder, output_filename)
os.makedirs(os.path.dirname(output_path), exist_ok=True)
plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
plt.close()
print(f"Image saved at: {output_path}")

auxembRec = All_BLymphocytes[All_BLymphocytes['RNR'] == 'R']
aux=All_BLymphocytes[All_BLymphocytes['Status'] == 'NonRegenerated']
aux=aux.assign(TDFindex=aux.index.values)
aux.set_index('TDFindex',inplace=True)
auxembNoRec=aux[aux['RNR'] == 'NR']

auxembRec.insert(2,"Lympho_label", list(Y_result_Rec), True)
auxembNoRec.insert(2,"Lympho_label", list(Y_result_NoRec), True)
combined_labels = pd.concat([df_Regenerated_BLympho['Lympho_label'], 
                             auxembRec['Lympho_label'], 
                             auxembNoRec['Lympho_label']])
combined_labels = combined_labels.sort_index()
All_BLymphocytes = All_BLymphocytes.assign(LymphoLabel=combined_labels)



def BLymphoPatient_t(name, t):
    colors = {'Mature': '#902747', 'Trans': '#E9AFC1', 'ProBPreB': '#A567CB','Unknown':'grey', np.nan: 'grey'}

    aux1 = All_BLymphocytes[All_BLymphocytes['name'] == name]
    aux2 = aux1[aux1['time'] == t]

    color_values = aux2['LymphoLabel'].map(colors).reset_index(drop=True)

    if color_values.isnull().any():
        raise ValueError("Hay etiquetas en 'LymphoLabel' que no tienen un color asignado en 'colors'.")

    plt.figure(figsize=(7, 7))
    plt.scatter(
        aux2['UMAP_BLympho_0'].values,
        aux2['UMAP_BLympho_1'].values,
        c=color_values.values,
        label=aux2['LymphoLabel'], 
        s=2
    )
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.title(f'B Lymphocytes {name} at {t}', fontsize=15)
    handles = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=col, markersize=3, label=f'{label}') 
        for label, col in colors.items()
    ]
    plt.xlabel('UMAP_BLympho_0', fontsize=15)
    plt.ylabel('UMAP_BLympho_1', fontsize=15)

    now = datetime.datetime.now()
    output_filename = f'B_Lymphocytes_{name}_at_time_{t}_{now.month}-{now.day}_at_{now.hour}_{now.minute}.png'
    output_path = os.path.join(ImagesFolder, 'Lymphocytes', output_filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')

    plt.show()
    print(f"image saved a: {output_path}")
    
    
def BLympho_group(RNR, t):
    aux1 = All_BLymphocytes[All_BLymphocytes['RNR'] == RNR]
    aux2 = aux1[aux1['time'] == t]
    Palette = 'crest' if RNR == 'NR' else 'rocket_r'

    i = 0
    marker = 'CD10'
    a = len(aux2.name.unique())
    b = int(np.sqrt(a))

    fig, axs = plt.subplots((a + b - 1) // b, b, figsize=(30, 30))
    fig.suptitle(f'B Lymphocytes of group {RNR} in {t}', fontsize=30)
    fig.subplots_adjust(top=0.9)
    fig.tight_layout(rect=[0, 0, 1, 0.96])

    for patient in aux2.name.unique():
        aux3 = aux2[aux2['name'] == patient]
        ax = axs[i // b, i % b]
        im = ax.scatter(
            aux3['UMAP_BLympho_0'], aux3['UMAP_BLympho_1'], 
            s=10, c=aux3[marker], cmap=sns.color_palette(Palette, as_cmap=True), vmin=-0.25, vmax=1.5
        )
        ax.grid(False)
        ax.set_title(f'CD10 {patient}', fontsize=20)
        fig.colorbar(im, ax=ax)
        i += 1

    for j in range(i, b * b):
        fig.delaxes(axs[j // b, j % b])

    now = datetime.datetime.now()
    output_filename = f'{RNR}_{t}_SeparatedPatients_{now.month}-{now.day}_at_{now.hour}_{now.minute}.png'
    output_path = os.path.join(ImagesFolder, 'Lymphocytes', output_filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.show()

    print(f"Imagen guardada en: {output_path}")
    
def BLympho_bytime(RNR):
    marker = 'CD45'
    a = 6
    b = int(np.sqrt(a))

    fig, axs = plt.subplots((a + b - 1) // b, b, figsize=(30, 30))
    fig.suptitle(f'B Lymphocytes of group {RNR} in {marker}', fontsize=30)
    fig.subplots_adjust(top=0.9)
    fig.tight_layout(rect=[0, 0, 1, 0.96])

    i = 0
    for t in ['t0', 't1', 't2', 't3', 't4', 'tr']:
        aux1 = All_BLymphocytes[All_BLymphocytes['RNR'] == RNR]
        aux2 = aux1[aux1['time'] == t]
        Palette = 'crest' if RNR == 'NR' else 'rocket_r'

        ax = axs[i // b, i % b]
        im = ax.scatter(
            aux2['UMAP_BLympho_0'], aux2['UMAP_BLympho_1'],
            s=10, c=aux2[marker], cmap=sns.color_palette(Palette, as_cmap=True), vmin=-0.25, vmax=1.5
        )
        ax.grid(False)
        ax.set_title(f'Intensity of {marker} in {t}', fontsize=20)
        fig.colorbar(im, ax=ax)
        i += 1

    for j in range(i, b * b):
        fig.delaxes(axs[j // b, j % b])

    now = datetime.datetime.now()
    output_filename = f'{RNR}_{marker}_AllPatients_{now.month}-{now.day}_at_{now.hour}_{now.minute}.png'
    output_path = os.path.join(ImagesFolder, 'Lymphocytes', output_filename)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    plt.savefig(output_path, bbox_inches="tight", pad_inches=1, transparent=True, facecolor="w", orientation='landscape')
    plt.show()

    print(f"Imagen guardada en: {output_path}")
    
TDF_EMB = pd.merge(TDF_EMB, All_BLymphocytes[['UMAP_BLympho_0', 'UMAP_BLympho_1', 'LymphoLabel']], on='TDFindex', how='outer')
TDF_EMB['LymphoLabel'].fillna('No_BLympho', inplace=True)
TDF_EMB['UMAP_BLympho_0'].fillna('No_BLympho', inplace=True)  
TDF_EMB['UMAP_BLympho_1'].fillna('No_BLympho', inplace=True)  

dataframes = {}
dataframes['TDF_EMB'] = TDF_EMB
dump(dataframes, 'TDF_EMB.joblib')